<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*このノートブックには、Jake VanderPlas による [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) からの抜粋が含まれています。コンテンツは利用可能です [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do).*

※テキストは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で、コードは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で公開しています。このコンテンツが役立つと思われる場合は、[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode) による作業のサポートを検討してください!*

<!--ナビゲーション-->
< [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb) | [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb) | [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/02.03-Computation-on-arrays-ufuncs.ipynb"><img align="left" src ="https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>


# NumPy 配列の計算: ユニバーサル関数

これまで、NumPy の基本的な基本事項について説明してきました。次のいくつかのセクションでは、NumPy が Python データ サイエンスの世界で非常に重要である理由について詳しく説明します。
つまり、データの配列を使用した最適化された計算への簡単で柔軟なインターフェイスを提供します。

NumPy 配列の計算は、非常に高速になることもあれば、非常に低速になることもあります。
高速化の鍵は、通常は NumPy の *ユニバーサル関数* (ufunc) によって実装される *ベクトル化された* 操作を使用することです。
このセクションは、NumPy の ufunc の必要性を動機付けます。これを使用すると、配列要素で繰り返し計算をより効率的に行うことができます。
次に、NumPy パッケージで利用できる最も一般的で便利な算術 ufunc の多くを紹介します。

## ループの遅さ

Python のデフォルトの実装 (CPython として知られています) は、いくつかの操作を非常に遅く実行します。
これは、部分的には言語の動的で解釈された性質によるものです。型が柔軟であるため、C や Fortran などの言語のように一連の操作を効率的なマシン コードにコンパイルすることはできません。
最近、この弱点に対処するためのさまざまな試みが行われています。よく知られている例は、Python のジャストインタイム コンパイル実装である [PyPy](http://pypy.org/) プロジェクトです。 Python コードをコンパイル可能な C コードに変換する [PyPy](http://pypy.org/) プロジェクト。 [PyPy](http://pypy.org/) プロジェクトは、Python コードのスニペットを高速な LLVM バイトコードに変換します。
これらにはそれぞれ長所と短所がありますが、3 つのアプローチのいずれも、標準の CPython エンジンの到達範囲と人気をまだ上回っていないと言っても過言ではありません。

Python の相対的な動作の遅さは、通常、多数の小さな操作が繰り返される状況で現れます。たとえば、配列をループして各要素を操作する場合などです。
たとえば、値の配列があり、それぞれの逆数を計算したいとします。
簡単なアプローチは次のようになります。

In [1]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

array([ 0.16666667,  1.        ,  0.25      ,  0.25      ,  0.125     ])

この実装は、たとえば C や Java のバックグラウンドを持つ人にとっては、おそらくかなり自然に感じるでしょう。
しかし、大きな入力に対してこのコードの実行時間を測定すると、この操作が非常に遅いことがわかります。
これを IPython の ``%timeit`` マジック ([Profiling and Timing Code](01.07-Timing-and-Profiling.ipynb) で説明) でベンチマークします。

In [2]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

1 loop, best of 3: 2.91 s per loop


これらの百万回の操作を計算して結果を保存するには、数秒かかります!
携帯電話でさえ処理速度がギガ FLOPS (つまり、1 秒あたり数十億回の数値演算) で測定される場合、これはほとんどばかげたほど遅いように見えます。
ここでのボトルネックは操作そのものではなく、CPython がループの各サイクルで実行しなければならない型チェックと関数ディスパッチであることがわかります。
逆数が計算されるたびに、Python はまずオブジェクトの型を調べ、その型に使用する正しい関数を動的に検索します。
代わりにコンパイルされたコードで作業していた場合、この型指定はコードが実行される前に認識され、結果をより効率的に計算できます。

## UFunc の紹介

多くの種類の操作に対して、NumPy は、まさにこの種の静的に型付けされ、コンパイルされたルーチンへの便利なインターフェイスを提供します。これは、*ベクトル化* 操作として知られています。
これは、配列に対して操作を実行するだけで実現でき、その操作は各要素に適用されます。
このベクトル化されたアプローチは、NumPy の基盤となるコンパイル済みレイヤーにループをプッシュするように設計されているため、実行が大幅に高速化されます。

次の 2 つの結果を比較します。

In [3]:
print(compute_reciprocals(values))
print(1.0 / values)

[ 0.16666667  1.          0.25        0.25        0.125     ]
[ 0.16666667  1.          0.25        0.25        0.125     ]


大きな配列の実行時間を見ると、Python ループよりも桁違いに速く完了することがわかります。

In [4]:
%timeit (1.0 / big_array)

100 loops, best of 3: 4.6 ms per loop


NumPy でのベクトル化された操作は、*ufuncs* を介して実装されます。その主な目的は、NumPy 配列の値に対して繰り返し操作をすばやく実行することです。
Ufunc は非常に柔軟です – スカラーと配列の間の操作を見る前に、2 つの配列間で操作することもできます。

In [5]:
np.arange(5) / np.arange(1, 6)

array([ 0.        ,  0.5       ,  0.66666667,  0.75      ,  0.8       ])

また、ufunc 操作は 1 次元配列に限定されません。多次元配列にも作用します。

In [6]:
x = np.arange(9).reshape((3, 3))
2 ** x

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]])

ufunc によるベクトル化を使用した計算は、特に配列のサイズが大きくなるにつれて、Python ループを使用して実装された対応する計算よりもほぼ常に効率的です。
Python スクリプトでこのようなループを見つけた場合は、ベクトル化された式に置き換えることができるかどうかを検討する必要があります。

## NumPy の UFunc の探索

Ufunc には 2 つの種類があります。単一の入力で動作する *単項 ufunc* と、2 つの入力で動作する *バイナリ ufunc* です。
ここでは、これら両方のタイプの関数の例を示します。

### 配列演算

NumPy の ufunc は、Python のネイティブ算術演算子を使用するため、非常に自然に使用できます。
標準的な足し算、引き算、掛け算、割り算はすべて使用できます。

In [7]:
x = np.arange(4)
print("x     =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 2)
print("x / 2 =", x / 2)
print("x // 2 =", x // 2)  # floor division

x     = [0 1 2 3]
x + 5 = [5 6 7 8]
x - 5 = [-5 -4 -3 -2]
x * 2 = [0 2 4 6]
x / 2 = [ 0.   0.5  1.   1.5]
x // 2 = [0 0 1 1]


否定のための単項 ufunc と、べき乗のための ``**`` 演算子と、剰余のための ``%`` 演算子もあります:

In [8]:
print("-x     = ", -x)
print("x ** 2 = ", x ** 2)
print("x % 2  = ", x % 2)

-x     =  [ 0 -1 -2 -3]
x ** 2 =  [0 1 4 9]
x % 2  =  [0 1 0 1]


さらに、これらは好きなように組み合わせることができ、操作の標準的な順序が尊重されます。

In [9]:
-(0.5*x + 1) ** 2

array([-1.  , -2.25, -4.  , -6.25])

これらの算術演算はそれぞれ、NumPy に組み込まれている特定の関数の便利なラッパーです。たとえば、``+`` 演算子は ``add`` 関数のラッパーです:

In [10]:
np.add(x, 2)

array([2, 3, 4, 5])

次の表に、NumPy で実装されている算術演算子を示します。

| |オペレーター |同等の ufunc |説明 |
|---------------|---------------------|---------------------------------------|
|``+`` |``np.add`` |加算 (例: ``1 + 1 = 2``) |
|``-`` |``np.subtract`` |減算 (例: ``3 - 2 = 1``) |
|``-`` |``np.negative`` |単項否定 (例: ``-2``) |
|``*`` |``np.multiply`` |乗算 (例: ``2 * 3 = 6``) |
|``/`` |``np.divide`` |除算 (例: ``3 / 2 = 1.5``) |
|``//`` |``np.floor_divide`` |フロア分割 (例: ``3 // 2 = 1``) |
|``**`` |``np.power`` |累乗 (例: ``2 ** 3 = 8``) |
|``%`` |``np.mod`` |モジュラス/剰余 (例: ``9 % 4 = 1``)|

さらに、ブール/ビット単位の演算子があります。これらについては、[Comparisons, Masks, and Boolean Logic](02.06-Boolean-Arrays-and-Masks.ipynb) で説明します。

＃＃＃ 絶対値

NumPy が Python の組み込みの算術演算子を理解するように、Python の組み込みの絶対値関数も理解します。

In [11]:
x = np.array([-2, -1, 0, 1, 2])
abs(x)

array([2, 1, 0, 1, 2])

対応する NumPy ufunc は ``np.absolute`` で、エイリアス ``np.abs`` の下でも利用できます:

In [12]:
np.absolute(x)

array([2, 1, 0, 1, 2])

In [13]:
np.abs(x)

array([2, 1, 0, 1, 2])

この ufunc は、絶対値が大きさを返す複雑なデータも処理できます。

In [14]:
x = np.array([3 - 4j, 4 - 3j, 2 + 0j, 0 + 1j])
np.abs(x)

array([ 5.,  5.,  2.,  1.])

### 三角関数

NumPy は多数の有用な ufunc を提供します。データ サイエンティストにとって最も有用なものは三角関数です。
角度の配列を定義することから始めます。

In [15]:
theta = np.linspace(0, np.pi, 3)

これで、これらの値に対していくつかの三角関数を計算できます。

In [16]:
print("theta      = ", theta)
print("sin(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))

theta      =  [ 0.          1.57079633  3.14159265]
sin(theta) =  [  0.00000000e+00   1.00000000e+00   1.22464680e-16]
cos(theta) =  [  1.00000000e+00   6.12323400e-17  -1.00000000e+00]
tan(theta) =  [  0.00000000e+00   1.63312394e+16  -1.22464680e-16]


値はマシンの精度内で計算されます。これが、ゼロであるべき値が常に正確にゼロになるとは限らない理由です。
逆三角関数も利用できます。

In [17]:
x = [-1, 0, 1]
print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

x         =  [-1, 0, 1]
arcsin(x) =  [-1.57079633  0.          1.57079633]
arccos(x) =  [ 3.14159265  1.57079633  0.        ]
arctan(x) =  [-0.78539816  0.          0.78539816]


### 指数と対数

NumPy ufunc で利用できるもう 1 つの一般的な操作のタイプは、指数関数です。

In [18]:
x = [1, 2, 3]
print("x     =", x)
print("e^x   =", np.exp(x))
print("2^x   =", np.exp2(x))
print("3^x   =", np.power(3, x))

x     = [1, 2, 3]
e^x   = [  2.71828183   7.3890561   20.08553692]
2^x   = [ 2.  4.  8.]
3^x   = [ 3  9 27]


指数の逆数である対数も利用できます。
基本的な ``np.log`` は自然対数を返します。 2 を底とする対数または 10 を底とする対数を計算したい場合は、これらも利用できます。

In [19]:
x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

x        = [1, 2, 4, 10]
ln(x)    = [ 0.          0.69314718  1.38629436  2.30258509]
log2(x)  = [ 0.          1.          2.          3.32192809]
log10(x) = [ 0.          0.30103     0.60205999  1.        ]


非常に小さな入力で精度を維持するのに役立つ特殊なバージョンもいくつかあります。

In [20]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

exp(x) - 1 = [ 0.          0.0010005   0.01005017  0.10517092]
log(1 + x) = [ 0.          0.0009995   0.00995033  0.09531018]


x が非常に小さい場合、これらの関数は生の np.log や np.exp を使用する場合よりも正確な値を返します。

### 特殊な ufunc

NumPy には、双曲線三角関数、ビットごとの算術演算、比較演算子、ラジアンから度への変換、丸めと剰余など、さらに多くの ufunc が用意されています。
NumPy のドキュメントに目を通すと、多くの興味深い機能が明らかになります。

より特殊化されたあいまいな ufunc のもう 1 つの優れたソースは、サブモジュール ``scipy.special`` です。
データに対してあいまいな数学関数を計算したい場合は、それが scipy.special に実装されている可能性があります。
すべてをリストするには関数が多すぎますが、次のスニペットは、統計のコンテキストで出てくる可能性のあるいくつかを示しています。

In [21]:
from scipy import special

In [22]:
# Gamma functions (generalized factorials) and related functions
x = [1, 5, 10]
print("gamma(x)     =", special.gamma(x))
print("ln|gamma(x)| =", special.gammaln(x))
print("beta(x, 2)   =", special.beta(x, 2))

gamma(x)     = [  1.00000000e+00   2.40000000e+01   3.62880000e+05]
ln|gamma(x)| = [  0.           3.17805383  12.80182748]
beta(x, 2)   = [ 0.5         0.03333333  0.00909091]


In [23]:
# Error function (integral of Gaussian)
# its complement, and its inverse
x = np.array([0, 0.3, 0.7, 1.0])
print("erf(x)  =", special.erf(x))
print("erfc(x) =", special.erfc(x))
print("erfinv(x) =", special.erfinv(x))

erf(x)  = [ 0.          0.32862676  0.67780119  0.84270079]
erfc(x) = [ 1.          0.67137324  0.32219881  0.15729921]
erfinv(x) = [ 0.          0.27246271  0.73286908         inf]


NumPy と ``scipy.special`` の両方で、さらに多くの ufunc が利用可能です。
これらのパッケージのドキュメントはオンラインで入手できるため、「gamma function python」の行に沿って Web 検索すると、通常、関連情報が見つかります。

## 高度な Ufunc 機能

多くの NumPy ユーザーは、すべての機能を学習することなく ufunc を利用しています。
ここでは、ufunc のいくつかの特殊な機能について概説します。

### 出力の指定

大規模な計算の場合、計算結果が格納される配列を指定できると便利な場合があります。
一時的な配列を作成するのではなく、これを使用して、計算結果を希望するメモリ位置に直接書き込むことができます。
すべての ufunc に対して、これは関数の ``out`` 引数を使用して行うことができます:

In [24]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y)
print(y)

[  0.  10.  20.  30.  40.]


これは、配列ビューでも使用できます。たとえば、指定した配列の他のすべての要素に計算の結果を書き込むことができます。

In [25]:
y = np.zeros(10)
np.power(2, x, out=y[::2])
print(y)

[  1.   0.   2.   0.   4.   0.   8.   0.  16.   0.]


代わりに ``y[::2] = 2 ** x`` と書いた場合、``2 ** x`` の結果を保持する一時的な配列が作成され、その後に 2 番目の配列が続きます。これらの値を ``y`` 配列にコピーする操作。
これは、このような小さな計算では大きな違いはありませんが、非常に大きな配列では、out 引数を慎重に使用することでメモリを大幅に節約できます。

### 集計

バイナリ ufunc の場合、オブジェクトから直接計算できる興味深い集計がいくつかあります。
たとえば、特定の操作で配列を *reduce* したい場合、任意の ufunc の ``reduce`` メソッドを使用できます。
reduce は、結果が 1 つだけになるまで、特定の操作を配列の要素に繰り返し適用します。

たとえば、add ufunc で reduce を呼び出すと、配列内のすべての要素の合計が返されます。

In [26]:
x = np.arange(1, 6)
np.add.reduce(x)

15

同様に、 ``multiply`` ufunc で ``reduce`` を呼び出すと、すべての配列要素の積が得られます:

In [27]:
np.multiply.reduce(x)

120

計算のすべての中間結果を保存したい場合は、代わりに ``accumulate`` を使用できます。

In [28]:
np.add.accumulate(x)

array([ 1,  3,  6, 10, 15])

In [29]:
np.multiply.accumulate(x)

array([  1,   2,   6,  24, 120])

これらの特定のケースでは、結果を計算するための専用の NumPy 関数 (``np.sum``、``np.prod``、``np.cumsum``、``np.cumprod``) があることに注意してください。これについては、[Aggregations: Min, Max, and Everything In Between](02.04-Computation-on-arrays-aggregates.ipynb) で説明します。

### 外装品

最後に、任意の ufunc は、「outer」メソッドを使用して、2 つの異なる入力のすべてのペアの出力を計算できます。
これにより、1 行で乗算表の作成などを行うことができます。

In [30]:
x = np.arange(1, 6)
np.multiply.outer(x, x)

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])

[Fancy Indexing](02.07-Fancy-Indexing.ipynb) で説明する ufunc.at と ufunc.reduceat メソッドも非常に役に立ちます。

ufunc のもう 1 つの非常に便利な機能は、さまざまなサイズと形状の配列間で操作できる機能です。これは、*ブロードキャスト* と呼ばれる一連の操作です。
このテーマは非常に重要であるため、セクション全体を取り上げます ([Computation on Arrays: Broadcasting](02.05-Computation-on-arrays-broadcasting.ipynb) を参照)。

## Ufuncs: もっと学ぶ

ユニバーサル関数 (使用可能な関数の完全なリストを含む) の詳細については、[NumPy](http://www.numpy.org) および [NumPy](http://www.numpy.org) ドキュメント Web サイトを参照してください。

[Help and Documentation in IPython](01.01-Help-And-Documentation.ipynb) で説明されているように、パッケージをインポートし、IPython のタブ補完とヘルプ (``?``) 機能を使用して、IPython 内から直接情報にアクセスすることもできることを思い出してください。

<!--ナビゲーション-->
< [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb) | [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb) | [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/02.03-Computation-on-arrays-ufuncs.ipynb"><img align="left" src ="https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>
